In [1]:
import json
import os
import pandas as pd
import geopandas as gpd
import fiona
from pyproj import CRS
from shapely.geometry import Point
fiona.drvsupport.supported_drivers['KML'] = 'rw'

def load_kml(path):
  df = gpd.GeoDataFrame()

  # iterate over layers
  for layer in fiona.listlayers(path):
      s = gpd.read_file(path, driver='KML', layer=layer)
      df = pd.concat([df, s], ignore_index=True)
  return df

In [2]:
survey = '230612_spurgepoly'

config = {
    'survey_res' : 1.32,
    'compute_array_sz' : 10,
    'gcp_res' : 200,
    'flight_plan_url' : f'https://raw.githubusercontent.com/samsoe/mpg_aerial_survey/main/surveys/{survey}/flightplan.kml',
    'photo_manifest_url' : f'https://storage.googleapis.com/mpg-aerial-survey/surveys/{survey}/data_collection/m3m/manifest.csv',
    'output_bucket' : f'mpg-aerial-survey/surveys/{survey}/post_process/odm/upstream',
    'gcp_editor_url' : f'https://raw.githubusercontent.com/samsoe/mpg_aerial_survey/main/surveys/{survey}/gcp_list.txt',
    'optimization_base_url' : f'https://raw.githubusercontent.com/samsoe/mpg_aerial_survey/main/surveys/{survey}/optimization/'
}              

local_path = '/Users/kdoherty/'
config_path = f'mpg_aerial_survey/surveys/{survey}'
out_path = os.path.join(local_path, config_path, 'config_file.json')

with open(out_path, 'w') as json_file:
    json.dump(config, json_file, indent=4)

In [5]:
flight_path = os.path.join(local_path, config_path, 'flightplan.kml')
gcps_path = os.path.join(local_path, 'mpg_aerial_survey/gcp_kmls/upland_gcps_200m.csv')

flight_poly = load_kml(flight_path)
gcps_points = pd.read_csv(gcps_path)

geometry = [Point(xy) for xy in zip(gcps_points.longitude, gcps_points.latitude)]
gcps_points = gpd.GeoDataFrame(gcps_points, geometry=geometry)
gcps_flight = gpd.sjoin(gcps_points, flight_poly, how='inner', op='within')

columns_to_keep = ['name', 'longitude', 'latitude', 'ellipsoidal_height']

# Filter the GeoDataFrame to retain only the specified columns
filtered_data = gcps_flight.filter(columns_to_keep)
new_columns = {col: col.replace('_', ' ') for col in filtered_data.columns}
filtered_data.rename(columns=new_columns, inplace=True)
gcps_out = os.path.join(local_path, config_path, 'gcps_roi.csv')

filtered_data.to_csv(gcps_out, index=False)

/opt/miniconda3/envs/ml/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/vp/1c12nhtx09ng562vszvlczthvwd07y/T/ipykernel_93914/3346547455.py:9: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  gcps_flight = gpd.sjoin(gcps_points, flight_poly, how='inner', op='within')
